<a href="https://colab.research.google.com/github/wallas999/Mis_Proyectos/blob/main/Elevators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Version 1.2.1
# [ES] 1 elevador, 2 lógicas de operación, todos al mismo destino: 1er piso
# [EN] 1 elevator, 2 operation logics, all same destination: 1st floor

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [5]:
def simular_scan(
    llamadas,
    piso_inicial,
    piso_min=1,
    piso_max=10,
    consumo_subida=4.0,
    consumo_bajada=1.0,
    consumo_parada=1.0,
    tiempo_por_piso=5,
    tiempo_parada=10
):
    piso_actual = piso_inicial
    consumo_total = 0.0
    tiempo_total = 0
    historial = []

    # Filtrar llamadas hacia arriba y hacia abajo
    llamadas_arriba = sorted([ll for ll in llamadas if ll[0] > piso_actual], key=lambda x: x[0])
    llamadas_abajo = sorted([ll for ll in llamadas if ll[0] < piso_actual], key=lambda x: x[0], reverse=True)

    # Agregar piso máximo y mínimo para imitar el barrido SCAN
    if piso_max > piso_actual and (piso_max, None) not in llamadas_arriba:
        llamadas_arriba.append((piso_max, None))
        llamadas_arriba.sort(key=lambda x: x[0])
    if piso_min < piso_actual and (piso_min, None) not in llamadas_abajo:
        llamadas_abajo.append((piso_min, None))
        llamadas_abajo.sort(key=lambda x: x[0], reverse=True)

    # --- SUBIDA ---
    # Primero se recogen todos los pasajeros subiendo (solo orígenes)
    for origen, _ in llamadas_arriba:
        pisos_mov = abs(origen - piso_actual)
        energia_mov = pisos_mov * consumo_subida
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa
        tiempo_etapa = pisos_mov * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual, "Hasta": origen, "Sube": True,
            "Distancia (pisos)": pisos_mov, "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total, "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = origen

    # Luego se entregan todos los pasajeros que van hacia arriba (destinos)
    destinos_arriba = sorted(set(d for _, d in llamadas_arriba if d is not None and d >= piso_actual))
    for destino in destinos_arriba:
        pisos_mov = abs(destino - piso_actual)
        energia_mov = pisos_mov * consumo_subida
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa
        tiempo_etapa = pisos_mov * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual, "Hasta": destino, "Sube": True,
            "Distancia (pisos)": pisos_mov, "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total, "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = destino

    # --- BAJADA ---
    # Recoger pasajeros bajando (orígenes)
    for origen, _ in llamadas_abajo:
        pisos_mov = abs(origen - piso_actual)
        energia_mov = pisos_mov * consumo_bajada
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa
        tiempo_etapa = pisos_mov * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual, "Hasta": origen, "Sube": False,
            "Distancia (pisos)": pisos_mov, "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total, "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = origen

    # Entregar pasajeros bajando (destinos)
    destinos_abajo = sorted(set(d for _, d in llamadas_abajo if d is not None and d <= piso_actual), reverse=True)
    for destino in destinos_abajo:
        pisos_mov = abs(destino - piso_actual)
        energia_mov = pisos_mov * consumo_bajada
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa
        tiempo_etapa = pisos_mov * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual, "Hasta": destino, "Sube": False,
            "Distancia (pisos)": pisos_mov, "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total, "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = destino

    return pd.DataFrame(historial), consumo_total, tiempo_total




def simular_look(
    llamadas,
    piso_inicial,
    consumo_subida=4.0,
    consumo_bajada=1.0,
    consumo_parada=1.0,
    tiempo_por_piso=5,
    tiempo_parada=10
):
    piso_actual = piso_inicial
    consumo_total = 0.0
    tiempo_total = 0
    historial = []

    def mover_ascensor(desde, hasta):
        nonlocal consumo_total, tiempo_total
        if desde == hasta:
            return
        sube = hasta > desde
        pisos_mov = abs(hasta - desde)
        energia_mov = pisos_mov * (consumo_subida if sube else consumo_bajada)
        energia_etapa = energia_mov + consumo_parada
        tiempo_etapa = pisos_mov * tiempo_por_piso + tiempo_parada
        consumo_total += energia_etapa
        tiempo_total += tiempo_etapa
        historial.append({
            "Desde": desde, "Hasta": hasta, "Sube": sube,
            "Distancia (pisos)": pisos_mov, "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total, "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })

    # Clasificar llamadas según dirección real del viaje
    llamadas_arriba = [(o, d) for o, d in llamadas if d > o]
    llamadas_arriba.sort(key=lambda x: x[0])  # ordenar por origen ascendente

    llamadas_abajo = [(o, d) for o, d in llamadas if d < o]
    llamadas_abajo.sort(key=lambda x: x[0], reverse=True)  # ordenar por origen descendente

    # === SUBIDA ===
    if llamadas_arriba:
        # Recoger pasajeros en orden ascendente
        for origen, _ in llamadas_arriba:
            mover_ascensor(piso_actual, origen)
            piso_actual = origen

        # Entregar pasajeros en orden ascendente de destino
        destinos_subida = sorted(set(d for _, d in llamadas_arriba))
        for destino in destinos_subida:
            mover_ascensor(piso_actual, destino)
            piso_actual = destino

    # === BAJADA ===
    if llamadas_abajo:
        # Recoger pasajeros en orden descendente
        for origen, _ in llamadas_abajo:
            mover_ascensor(piso_actual, origen)
            piso_actual = origen

        # Entregar pasajeros en orden descendente de destino
        destinos_bajada = sorted(set(d for _, d in llamadas_abajo), reverse=True)
        for destino in destinos_bajada:
            mover_ascensor(piso_actual, destino)
            piso_actual = destino

    return pd.DataFrame(historial), consumo_total, tiempo_total







In [6]:
# Parámetros de prueba
piso_inicial = 3
# Ahora las llamadas son tuplas (origen, destino)
llamadas = [(2, 1), (4, 1), (6, 1)]

# Ejecutar simulación SCAN
df_historial_scan, total_energia_scan, total_tiempo_scan = simular_scan(llamadas, piso_inicial)
# Crear lista de pisos recorridos para SCAN
pisos_recorrido_scan = [piso_inicial] + [fila["Hasta"] for _, fila in df_historial_scan.iterrows()]

# Ejecutar simulación LOOK
df_historial_look, total_energia_look, total_tiempo_look = simular_look(llamadas, piso_inicial)
# Crear lista de pisos recorridos para LOOK
pisos_recorrido_look = [piso_inicial] + [fila["Hasta"] for _, fila in df_historial_look.iterrows()]

# --- Mostrar resultados SCAN ---
df_mod_scan = df_historial_scan.copy()
df_mod_scan.columns = [
    "Desde",
    "Hasta",
    "Sube",
    "Distancia (pisos)",
    "Energía etapa (Wh)",
    "Energía acumulada (Wh)",
    "Tiempo etapa (s)",
    "Tiempo acumulado (s)"
]

print("=== Resultados SCAN ===")
display(df_mod_scan)
print(f"Energía total: {total_energia_scan:.2f} Wh")
print(f"Tiempo total: {total_tiempo_scan} segundos\n")

# --- Mostrar resultados LOOK ---
df_mod_look = df_historial_look.copy()
df_mod_look.columns = [
    "Desde",
    "Hasta",
    "Sube",
    "Distancia (pisos)",
    "Energía etapa (Wh)",
    "Energía acumulada (Wh)",
    "Tiempo etapa (s)",
    "Tiempo acumulado (s)"
]

print("=== Resultados LOOK ===")
display(df_mod_look)
print(f"Energía total: {total_energia_look:.2f} Wh")
print(f"Tiempo total: {total_tiempo_look} segundos\n")



# Ahora la animación
piso_min = min(min(pisos_recorrido_scan), min(pisos_recorrido_look))
piso_max = max(max(pisos_recorrido_scan), max(pisos_recorrido_look))

plt.ioff()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 4), sharey=True)

for ax in (ax1, ax2):
    ax.set_ylim(piso_min - 1, piso_max + 1)
    ax.set_xlim(0, 1)
    ax.set_xticks([])
    ax.set_ylabel("Piso")

ax1.set_title("Elevator SCAN")
ax2.set_title("Elevator LOOK")

for piso in range(piso_min, piso_max + 1):
    ax1.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')
    ax2.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')

punto_scan, = ax1.plot([], [], 's', markersize=20, color='blue')
texto_energia_scan = ax1.text(0.5, piso_min - 1, '', ha='center')

punto_look, = ax2.plot([], [], 's', markersize=20, color='green')
texto_energia_look = ax2.text(0.5, piso_min - 1, '', ha='center')

max_frames = max(len(pisos_recorrido_scan), len(pisos_recorrido_look))

def init():
    punto_scan.set_data([], [])
    texto_energia_scan.set_text('')
    punto_look.set_data([], [])
    texto_energia_look.set_text('')
    return punto_scan, texto_energia_scan, punto_look, texto_energia_look

def actualizar(frame):
    if frame < len(pisos_recorrido_scan):
        piso_s = pisos_recorrido_scan[frame]
        punto_scan.set_data([0.5], [piso_s])
    else:
        piso_s = pisos_recorrido_scan[-1]
        punto_scan.set_data([0.5], [piso_s])

    if frame < len(pisos_recorrido_look):
        piso_l = pisos_recorrido_look[frame]
        punto_look.set_data([0.5], [piso_l])
    else:
        piso_l = pisos_recorrido_look[-1]
        punto_look.set_data([0.5], [piso_l])

    return punto_scan, texto_energia_scan, punto_look, texto_energia_look

ani = animation.FuncAnimation(
    fig, actualizar, frames=max_frames,
    init_func=init, blit=True, interval=1000, repeat=False
)

HTML(ani.to_jshtml())


=== Resultados SCAN ===


,Desde,Hasta,Sube,Distancia (pisos),Energía etapa (Wh),Energía acumulada (Wh),Tiempo etapa (s),Tiempo acumulado (s)
0,3,4,True,1,5.0,5.0,15,15
1,4,6,True,2,9.0,14.0,20,35
2,6,10,True,4,17.0,31.0,30,65
3,10,2,False,8,9.0,40.0,50,115
4,2,1,False,1,2.0,42.0,15,130
5,1,1,False,0,1.0,43.0,10,140


Energía total: 43.00 Wh
Tiempo total: 140 segundos

=== Resultados LOOK ===


,Desde,Hasta,Sube,Distancia (pisos),Energía etapa (Wh),Energía acumulada (Wh),Tiempo etapa (s),Tiempo acumulado (s)
0,3,6,True,3,13.0,13.0,25,25
1,6,4,False,2,3.0,16.0,20,45
2,4,2,False,2,3.0,19.0,20,65
3,2,1,False,1,2.0,21.0,15,80


Energía total: 21.00 Wh
Tiempo total: 80 segundos

